In [1]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph( 'model/.meta' )
    saver.restore( sess, "model/" )
    
    graph = tf.get_default_graph()
    file_dir = 'guangdong_round1_test_a_20180916'
    
    all_image_name = []
    all_pred = []
    
    for i in range( 440 ):
        image_name = str( i ) + '.jpg'
        file_name =  file_dir + '/' + image_name
        image = tf.cast( file_name, tf.string )
        image_contents = tf.read_file( image )
        image = tf.image.decode_jpeg(image_contents, channels=3) 
        image = tf.image.resize_image_with_crop_or_pad(image, 384, 384)
        #image = tf.expand_dims( image, 0 )
        image = tf.image.per_image_standardization(image)
        image_end = tf.reshape( image, [1, 384, 384, 3] )
        
        sess.run( image_end )
        feed_dict = { "x:0":image_end.eval(), "keep_prob:0":1 }
        pred = graph.get_tensor_by_name( "predictions:0" )
        prediction = sess.run( pred, feed_dict )
        all_image_name.append( image_name )
        all_pred.append( prediction[0] )
        if i % 10 == 0:
            print( i )

In [ ]:
dataframe = pd.DataFrame( {'1': all_image_name, '2': all_pred} )
dataframe.to_csv( "test.csv" , index=False, sep = ',' )